<a href="https://colab.research.google.com/github/MichaelKru92/Projekt-ML-Modelierung/blob/main/Schritt_3_Kurze_Datenexploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ein erster Blick auf die Daten**

Der folgenden Funktion wird eine Liste mit den Dateinamen der entpackten zip-Datei übergeben.

Es erfolgt eine erste kurze Exploration der Daten:

*   Darstellung der ersten 10 Zeilen jeder Tabelle
*   Ausgabe der Datentypen, Spaltennamen und Nullwerte

In [ ]:
import pandas as pd
import numpy as np
import os

# Retrieve the environment variable or default to the current directory
ROHDATEN_LOCAL_PATH = os.getenv('ROHDATEN_LOCAL_PATH', 'Rohdaten(CSV)')
CSV_PATH = ROHDATEN_LOCAL_PATH  # Use the environment variable directly

CSV_TABLES = ["careplans.csv", "conditions.csv", "devices.csv", "disease.csv", "encounters.csv", "imaging_studies.csv", "immunizations.csv",
              "medications.csv", "observations.csv", "organizations.csv", "patients.csv", "payer_transitions.csv", "payers.csv", "procedures.csv",
              "providers.csv", "supplies.csv"]

def load_csv_to_df_overview(csv_tables=CSV_TABLES, csv_path=CSV_PATH):
    for i in csv_tables:
        csv_table_path = os.path.join(csv_path, i)
        print(f"\n\nDie ersten 10 Zeilen von Tabelle {i}:")
        display(pd.read_csv(csv_table_path).head())
        print(f"\n\nMetadaten zu Tabelle {i}:\n")
        pd.read_csv(csv_table_path).info()
        print(f"\n\nNullwerte in Tabelle {i}:\n", pd.read_csv(csv_table_path).isnull().sum())


load_csv_to_df_overview()

**Zusammenfassung der Auswertung**

Als möglicherweise für eine ML-Auswertung zur Vorhersage von Gesundheitskosten beeinträchtigend zeigt sich im Allgemeinen Folgendes:
*   Spalten START und STOP Datumswerte als object gespeichert sind
*   Die Spalte CODE anthaelt einen SNOMED Code und ist als float gespeichert
*   Es existieren fehelende Einträge (Null-Werte). Betroffen sind die Spalte STOP, REASONCODE,    REASONDESCRIPTION in mehreren Tabellen, sowie ENCOUNTER-ID und UNITS in der Encounter-Tabelle

Insgesamt finden sich 135 Patienten in der Patients-Tabelle. Demgegenüber stehen 7768 Einträge in der Encounter-Tabelle. Daraus laesst sich vermuten, dass entweder mehrere Fälle pro Patient existieren oder Duplikate vorliegen.

Letzteres wird wie folgt eruiert:


In [ ]:
df_encounter = pd.read_csv(os.path.join(CSV_PATH, "encounters.csv"))
df_patients = pd.read_csv(os.path.join(CSV_PATH, "patients.csv"))
df_medications = pd.read_csv(os.path.join(CSV_PATH, "medications.csv"))
df_observations = pd.read_csv(os.path.join(CSV_PATH, "observations.csv"))
df_immunizations = pd.read_csv(os.path.join(CSV_PATH, "immunizations.csv"))
df_procedures = pd.read_csv(os.path.join(CSV_PATH, "procedures.csv"))


def check_for_duplicates(df, column):
  df[column] = df[column].apply(str)
  dup_df = df.duplicated(subset= [column])
  print (f"Anzahl der Duplikate in Spalte {column}:", dup_df.sum())
  print (df[dup_df].head(10))

check_for_duplicates(df_patients, "Id")
check_for_duplicates(df_medications, "ENCOUNTER")
check_for_duplicates(df_immunizations, "ENCOUNTER")
check_for_duplicates(df_observations, "ENCOUNTER")
check_for_duplicates(df_procedures, "ENCOUNTER")